# Знакомство с `NumPy`

Первая сторонняя библиотека, с которой мы познакомимся --- [NumPy](https://numpy.org/) (Numerical Python).

Библиотека `NumPy` предоставляет доступ к многомерным массивам однородных данных (одного типа, обычно числа), подобным массивам `C/C++`, и к методам их обработки. На самом деле, под капотом эти самые массивы реализованы с помощью динамических массивов `C/C++`, которые в отличие от списков, хранят непосредственно данные, а не указатели на них. 

## Зачем нужен `NumPy`

Необходимость в специальной библиотеке для работы с большими массивами чисел возникает из-за скорости работы `python`. За гибкость, выразительность, динамическую типизацию и многие другие достоинства, `python` приходится платить скоростью: код написанные на `python`, практически всегда будет работать медленнее, чем аналогичный код, написанный на `C/C++`. Проще всего продемонстрировать это на примере с циклами. 


````{tabbed} c++
```c++
#include <iostream>
#include <chrono>
#include <vector>
#include <random>
#define N 10000000

int main(){
    // Готовим генератор случайных чисел в промежутке от -1 до 1
    std::random_device rd;
    std::mt19937 gen(rd());
    std::uniform_real_distribution<> dis(-1.0, 1.0);
    
    // Создаём массивы случайных чисел
    std::vector<double> a(N), b(N), c(N);
    for(int i=0; i < N; ++i){
        a[i] = dis(gen);
        b[i] = dis(gen);
    }
	
    // Измеряем время сложения векторов
    auto t1 = std::chrono::high_resolution_clock::now();
    
    for(int i = 0; i < N; ++i)
        c[i] = a[i] + b[i];
        
    auto t2 = std::chrono::high_resolution_clock::now();
    
    auto ms_int = std::chrono::duration_cast<std::chrono::milliseconds>(t2 - t1);
    std::cout << ms_int.count() << " ms";
}
```
````
````{tabbed} python
```python
from time import perf_counter
from random import uniform


N = 10_000_000

# Создаём списки случайных чисел
a = [uniform(-1, 1) for i in range(N)] # новый синтаксис, список случайных чисел
b = [uniform(-1, 1) for i in range(N)]
c = [0.] * N 

# Измеряем время сложения этих списков
t1 = perf_counter()

for i in range(N):
    c[i] = a[i] + b[i]
        
t2 = perf_counter()
print(f"{(t2 - t1) * 1000} ms") # time() - время в секундах
```
````



```{note}
Документацию по модулю `time` можно найти [по этой ссылке](https://docs.python.org/3/library/time.html), а по модулю `random` [по этой ссылке](https://docs.python.org/3/library/random.html). 
```

Выше приведено содержимое двух файлов с исходным кодом, каждый из которых складывает два вектора чисел размерности $10^8$. 
1. Первый файл на языке `C++` использует для представления векторов в программе [std::vector](https://en.cppreference.com/w/cpp/container/vector). 
2. Второй файл на языке `python` использует для представления векторов в программе списки.

Эти файлы располагаются [по ссылке](/scripts/numpy/) и могут быть запущены следующему командами в командной строке.
```sh
g++ loops.cpp -O2 -o loops
.\loops.exe
python loops.py
```

В среднем я получил следующие результаты.
|`C++`   |`python`|
| :---:  | :---:  |
|  25 ms | 1700 ms|

```{note}
Вообще говоря, на эти цифры может повлиять огромное количество факторов, в том числе и случайных. Тем не менее разница настолько явная, что качественная картина ясна: циклы в `C/C++` работают заметно быстрее, чем в `python`.
```

"Среднее" приложение на python не сталкивается с обработкой больших объемов данных и обычно разница в производительности компенсируется затраченным на реализацию алгоритма временем. Но научные вычисления очень часто представляют собой "программы по перемолке чисел" и такое замедление существенно. 

Чтобы совместить скорость компилируемого языка и удобство `python` многие библиотеки поставляются вместе со скомпилированными модулями, написанными на `C/++` (или другом компилируемом языке), а в `python` "прокидывают" интерфейс для взаимодействия с ними. 




## Установка NumPy

В anaconda numpy установлен по умолчанию, а установить его с помощью `PyPI` можно следующим образом: 

```zsh
python -m pip install numpy
```

Установив NumPy, чтобы начать им пользоваться, необходимо его импортировать.

In [1]:
import numpy as np

Библиотека `NumPy` используется настолько часто и настолько многими, что сложилась традиция импортировать `NumPy` с псевдонимом `np`. Любой программист, увидев где-то в коде выражение вида `np.some_method()` догадается, что вызывается метод `some_method` из библиотеки `NumPy` и ему не потребуется искать глазами определение объекта `np`, чтобы понять, что скрывается за этим именем.

## О массивах `NumPy`. Тип данных, размерность, оси и форма.

В ячейке ниже создаётся массив `NumPy` из случайных чисел. Можете пока не вникать, как конкретно он создаётся. Важно лишь понять, что имя `array` связывается с каким-то массивом `NumPy`.

In [6]:
array = np.random.random(size=(2, 3))
print(array)

[[0.78691594 0.58196961 0.86767001]
 [0.63891866 0.30963327 0.28967785]]


Массивы в библиотеке представляются типом [numpy.ndarray](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html) (**n** **d**imensional **array**, n-мерный массив).

In [17]:
print(type(array))

<class 'numpy.ndarray'>


Тип `ndarray` является типизированным контейнером данных, т.к. он предназначен для хранения данных одного и того же типа данных. Тип данных массива хранится в атрибуте `dtype` (**d**ata **t**ype). Этих типов может быть много, но большинство из них --- стандартные числовые типы `C/C++`: `np.bool8`, `np.int8`, `np.int16`, `np.int32`, `np.int64`, `np.unit8`, `np.unit16`, `np.unit32`, `np.uint64`, `np.float16`, `np.float32`, `np.float64`, `np.complex64`, `np.complex128` ([подробнее про типы](https://numpy.org/doc/stable/user/basics.types.html)). 

```{note}
Целые числа в массивах `NumPy` введут себя совсем не как тип `int` из `python`: они всегда занимают определенное количество байт и могут переполняться, т.е. совпадают с целочисленными типами из `C/C++`. 
```

```{note}
Вообще говоря, в массивах `NumPy` можно хранить и объекты произвольного типа, но тогда вместо самих объектов, в массиве будут храниться ссылки на них. Чаще всего массивы `NumPy` используются все же для хранения чисел.
```

In [11]:
print(array.dtype)

float64


Исходя из значения атрибута `dtype` массива `array` можно понять, что он хранит в себе 64-битные числа с плавающей запятой.

Ещё `NumPy` массивы хранят количество своих измерений в атрибуте `ndim` (**n**umber of **d**imensions). Массивы `NumPy` могут быть любой размерности: одномерные массивы часто используются для представления векторов, двумерные --- для представления матриц и таблиц. 

In [12]:
array.ndim

2

Исходя из значения атрибута `ndim` массива `array` видим, что он является двухмерным.

Ещё обязательный атрибут массивов `NumPy` --- форма (`shape`) --- размеры массива вдоль каждого измерения. Эти измерения называются осями (`axis` в ед. числе и `axes` в мн. числе). Общее количество элементов в массиве (произведение количеств элементов вдоль каждой из осей) хранится в атрибуте `size`.

In [14]:
array.shape

(2, 3)

Атрибут `shape` массива `array` говорит нам, что у него есть 2 элемента вдоль первого измерения и 3 вдоль второго. Иными словами можно сказать, что это таблица из двух строк и трех столбцов.

Все массивы должны быть "прямоугольными", т.е. не может быть матрицы со строками разных длин. 

Картинка ниже наглядно иллюстрирует смысл всех только-что введенных понятий.
```{figure} /_static/lecture_specific/numpy/ndarrays.png
```

## Создание массивов

Создавать массивы `NumPy` --- `ndarray` --- можно огромным количеством образом, но для их создания редко используется конструктор самого типа. Вместо этого гораздо чаще используются другие методы. Например, метод [numpy.array](https://numpy.org/doc/stable/reference/generated/numpy.array.html) конструирует `ndarray` из других объектов. 

Чаще всего он используется для создания массивов из `python` списков чисел. Тип данных выводится как самый общий тип данных, если он не указан явно именованным параметром `dtype`.  

Ниже создаётся одномерный массив из списка четырёх чисел. При этом тип данных явно не указывается и в итоге получается массив чисел с плавающей точкой (`float64`), т.к. в исходном списке были числа типа `int` и `float`.

In [21]:
a1D = np.array([1., 2, 3, 4]) 
print(repr(a1D)) 
print(f"{a1D.dtype=}")
print(f"{a1D.ndim=}")
print(f"{a1D.shape=}")

array([1., 2., 3., 4.])
a1D.dtype=dtype('float64')
a1D.ndim=1
a1D.shape=(4,)


В ячейке ниже демонстрируется, как создаются двухмерные массивы. Для этого её на вход передаются списки списков, где каждый вложенный список --- строка будущей таблицы. Т.к. внутри всех этих списков находятся только целые числа, то итоговый тип массива тоже целочисленный.

In [22]:
a2D = np.array([[1, 2], [3, 4]]) 
print(repr(a2D)) 
print(f"{a2D.dtype=}")
print(f"{a2D.ndim=}")
print(f"{a2D.shape=}")

array([[1, 2],
       [3, 4]])
a2D.dtype=dtype('int32')
a2D.ndim=2
a2D.shape=(2, 2)


Трехмерные массивы создаются по аналогии. В ячейке ниже приводится пример. Обратите внимание, что все числа в передаваемом списке списков списков целочисленного типа, но результирующий массив получается типа `float32`, т.к. этот тип явно указан при создании массива.

In [24]:
a3D = np.array([[[1, 2], [3, 4]], [[5, 6], [7, 8]]], dtype=np.float32) 
print(repr(a3D)) 
print(f"{a3D.dtype=}")
print(f"{a3D.ndim=}")
print(f"{a3D.shape=}")

array([[[1., 2.],
        [3., 4.]],

       [[5., 6.],
        [7., 8.]]], dtype=float32)
a3D.dtype=dtype('float32')
a3D.ndim=3
a3D.shape=(2, 2, 2)


Так же есть ряд встроенных функций, создающих массивы. Каждая из них принимает в качестве опционального аргумента `dtype`.

|Метод | Аргументы | Описание|
| :---: | --- | :---: |
|[np.arange](https://numpy.org/doc/stable/reference/generated/numpy.arange.html#numpy.arange)| `start`, `stop`, `end` | Аналог `np.array(range)`, но можно использовать `float` в качестве аргументов |
|[np.linspace](https://numpy.org/doc/stable/reference/generated/numpy.linspace.html#numpy.linspace)| `start`, `stop`, `num`, `endpoint=True` | Разбиение отрезка [`start`, `stop`] на `num` отрезков, если передать в качестве `endpoint=False`, то последняя точка не включается |
|[np.zeros](https://numpy.org/doc/stable/reference/generated/numpy.zeros.html#numpy.zeros) | `shape` | Массив заданной формы из нулей |
|[np.ones](https://numpy.org/doc/stable/reference/generated/numpy.ones.html#numpy.ones)| `shape` | Массив заданной формы из единиц |
|[np.random](https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.random.html#numpy.random.Generator.random)|| Не метод, а подмодуль, содержащий методы создания массивов случайных значений | 
|[np.eye](https://numpy.org/doc/stable/reference/generated/numpy.eye.html#numpy.eye) |`n`, `m` | Если `m` не указан, то квадратная единичная матрица размера `nxn`, иначе нулевая матрица размера `nxm` с единицами на диагонали|
|[np.diag](https://numpy.org/doc/stable/reference/generated/numpy.diag.html#numpy.diag)|`v`, `k=0` | Матрица с вектором `v` на главной диагонали (`k=0`) или на `k`-й диагонали |
|[np.vand](https://numpy.org/doc/stable/reference/generated/numpy.vander.html#numpy.vander)|`x`, `N=None`, `increasing=False`| Матрица Вандермонда из вектора `x`|

Продемонстрируем ключевые из них. Массив из нулей создаётся методом [np.zeros](https://numpy.org/doc/stable/reference/generated/numpy.zeros.html#numpy.zeros), который на вход принимает форму массива.

In [30]:
zeros = np.zeros(shape=(2, 2))
print(zeros)

[[0. 0.]
 [0. 0.]]


Функция [np.ones](https://numpy.org/doc/stable/reference/generated/numpy.ones.html#numpy.ones) делает тоже самое, но создаёт массив из единиц.

In [34]:
ones = np.ones(shape=(2, 2))
print(ones)

[[1. 1.]
 [1. 1.]]


Функция [np.eye](https://numpy.org/doc/stable/reference/generated/numpy.eye.html#numpy.eye) часто создаётся для создания единичных матриц.

In [33]:
eye = np.eye(2)
print(eye)

[[1. 0.]
 [0. 1.]]


Функция [np.diag](https://numpy.org/doc/stable/reference/generated/numpy.diag.html#numpy.diag) конструирует диагональную матрицу, используя для заполнения диагонали элементы из переданного ей массива.

In [35]:
diag = np.diag([3.14, 42])
print(diag)

[[ 3.14  0.  ]
 [ 0.   42.  ]]




Кроме того, существует ряд методов, позволяющих создавать массивы из других массивов.

|Метод | Описание|
| :--- | ---: |
|[np.hstack](https://numpy.org/doc/stable/reference/generated/numpy.hstack.html#numpy.hstack)| Присоединение массивов вдоль строк|
|[np.vstack](https://numpy.org/doc/stable/reference/generated/numpy.vstack.html#numpy.vstack)| Присоединение массивов вдоль столбцов |
|[np.dstack](https://numpy.org/doc/stable/reference/generated/numpy.dstack.html#numpy.dstack)| Присоединение массивов "в глубину" |
|[np.block](https://numpy.org/doc/stable/reference/generated/numpy.hstack.html#numpy.hstack) | Составление массива из блоков |

Продемонстрируем метод [np.block](https://numpy.org/doc/stable/reference/generated/numpy.hstack.html#numpy.block), который позволяет создавать массивы из блоков, в качестве которых могут выступать другие массивы.

In [37]:
print(np.block([[zeros, ones], [eye, diag]]))

[[ 0.    0.    1.    1.  ]
 [ 0.    0.    1.    1.  ]
 [ 1.    0.    3.14  0.  ]
 [ 0.    1.    0.   42.  ]]


Более подробно о создании массивов `NumPy` по [ссылке](https://numpy.org/doc/stable/user/basics.creation.html).

## Арифметические операции над массивами `NumPy`

Все основные арифметические операторы (`+`, `-`, `*`, `/`, `//`, `%`, `**`) перегружены для работы с массивами. Определим скаляры и векторы, чтобы разобраться, какие действия они позволяют выполнять.

In [9]:
x = np.array([0, 1, 2])
y = np.array([7, 13, 42])
alpha = np.pi 
beta = 3

A = np.eye(3)
B = np.ones((3, 3))

Если в арифметическом выражении участвуют скаляр и массив, то результатом является массив значений поэлементного применения указанной арифметической операции с скаляром. Т.е., можно, например, умножить массив на число или даже сложить массив с числом.

In [10]:
print(alpha * x)
print(alpha + x)
print(alpha - x)
print(alpha / y)
print(y / alpha)
print(y // beta)
print(y % beta)
print(x ** beta)

[0.         3.14159265 6.28318531]
[3.14159265 4.14159265 5.14159265]
[3.14159265 2.14159265 1.14159265]
[0.44879895 0.24166097 0.07479983]
[ 2.2281692   4.13802852 13.36901522]
[ 2  4 14]
[1 1 0]
[0 1 8]


Если в выражении участвует два массива, то они должны быть одинаковой формы, а арифметическая операция применяется поэлементно.


```{note}
Важно запомнить, что все арифметические операторы выполняют поэлементное преобразование. Это в каком-то смысле является стандартным поведением python и многие другие действия `NumPy` выполняет поэлементно.  Далее рассматривается матричное умножение --- принципиально не поэлементная операция.  
```

In [11]:
print(x + y)
print(y - x)
print(x * y)
print(x / y)
print(x // y)
print(x % y)
print(x ** y)

[ 7 14 44]
[ 7 12 40]
[ 0 13 84]
[0.         0.07692308 0.04761905]
[0 0 0]
[0 1 2]
[0 1 0]


## Матричное умножение

Оператор `*` совершает поэлементное умножение, в том числе и для для двухмерных массивов. Если хочется вложить в двухмерные массив смысл матрицы из линейной алгебры и умножить их как матрицы, то есть два пути:

a) Использовать оператор `@`:

In [12]:
print(A, B, A * B, A @ B, sep="\n" + "-" * 10 + "\n")

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
----------
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
----------
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
----------
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]


b) Воспользоваться специальным типом `np.matrix` предназначенным для хранения именно двухмерных матриц.

In [13]:
A_matrix = np.matrix(A)
B_matrix = np.matrix(B)
print(repr(A_matrix), repr(B_matrix), repr(A_matrix * B_matrix), sep="\n" + "-" * 20 + "\n")

matrix([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])
--------------------
matrix([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
--------------------
matrix([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])


`np.matrix` наследуется от `ndarray` и отличается от него в следующем: 
 - `np.matrix` обязательно двухмерный массив, а `ndarray` имеет произвольную размерность;
 - `*` имеет смысл поэлементного управления для `ndarray` и матричного умножения для `np.matrix`;
 - `**` --- поэлементное возведение в степень для `ndarray` и матричное возведение для `np.matrix`;
 - `np.matrix.H` и `np.matrix.I` возвращают сопряженную и обратную матрицу;


```{warning}
Создатели NumPy рекомендуют использовать `ndarray`, и не дают гарантию, что в одном из будущих обновлений `np.matrix` не пропадут из библиотеки.
```
## Индексация

Индексация одномерных массивов `ndarray` осуществляется очень похоже на индексацию последовательностей `python` (списков, например).

In [14]:
print(a1D)
print(a1D[0], a1D[-1])

[1. 2. 3. 4.]
1.0 4.0


Индексация многомерных осуществляется несколько с новым синтаксисом.

In [15]:
print(a2D, end="\n" + "-"*10 + "\n")
print(a2D[0], end="\n" + "-"*10 + "\n")
print(a2D[-1, -1], a2D[-1][-1])

[[1 2]
 [3 4]]
----------
[1 2]
----------
4 4


In [16]:
print(a3D, end="\n" + "-"*10 + "\n")
print(a3D[0], end="\n" + "-"*10 + "\n")
print(a3D[-1, -1], a3D[-1][-1], end="\n" + "-"*10 + "\n")
print(a3D[-1, -1, -1])

[[[1. 2.]
  [3. 4.]]

 [[5. 6.]
  [7. 8.]]]
----------
[[1. 2.]
 [3. 4.]]
----------
[7. 8.] [7. 8.]
----------
8.0


## Изменение формы массива

Можно изменять размерность и форму массива, но с условием, что общее количество элементов остается (`size`) неизменным.
- [np.reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html#numpy.reshape) возвращает новый объект с измененной формой. 
- [numpy.ndarray.resize](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.resize.html#numpy.ndarray.resize) изменяет сам объект.
- [numpy.ravel](https://numpy.org/doc/stable/reference/generated/numpy.ravel.html#numpy.ravel) возвращает новый одномерный массив с теми же данными.

In [17]:
A = np.arange(24)
print(A)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]


In [18]:
B = A.reshape(4, 6)
print(B)

[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]]


In [19]:
C = A.reshape(4, 3, 2)
print(C)

[[[ 0  1]
  [ 2  3]
  [ 4  5]]

 [[ 6  7]
  [ 8  9]
  [10 11]]

 [[12 13]
  [14 15]
  [16 17]]

 [[18 19]
  [20 21]
  [22 23]]]


In [20]:
A.resize(2, 12)
print(A)

[[ 0  1  2  3  4  5  6  7  8  9 10 11]
 [12 13 14 15 16 17 18 19 20 21 22 23]]


In [21]:
A = np.ravel(A) # или A = A.ravel()
print(A)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]


## View. Разделяемое владение данными и копирование

На самом деле массивы `ndarray` устроены чуть сложнее. Массивы `ndarray` могут разделять одни и те же данные между собой. Массивы можно разделить на два вида: `base` и `view`. `view` --- `ndarray`, который не имеет собственных данных, а ссылается на данные внутри массива `base`. `base` массив владеет данными, `view` лишь позволяет по-другому на них посмотреть.

Каждый `ndarray` представляет собой объект, который хранит не только указатель на данные (нечто очень похожее на `C` массив), но еще и метаданные, говорящие массиву, каким образом интерпретировать эти данные.

Рассмотрим пример. Создадим массив `A` и убедимся, что он владеет своими данными.

In [22]:
A = np.arange(6)
print(A)
print("Флаги массива A: \n", A.flags)

[0 1 2 3 4 5]
Флаги массива B: 
   C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False



Мы создали массив (`ndarray`) размера 6, который владеет данными `[0, 1, 2, 3, 4, 5]` типа `np.int`. Созданный массив мы связали с именем `A`.

```{figure} /_static/lecture_specific/numpy/base_a.svg
```

Теперь создадим новый массив `B` путем изменения формы массива `A` методом `reshape`. Убедимся, что он использует те же данные.

In [23]:
B = A.reshape(2, 3)
print(B)
print("Массив B пользуется данными массива A?", B.base is A)
print("Флаги массива B: \n", B.flags)

[[0 1 2]
 [3 4 5]]
Массив B пользуется данными массива A? True
Флаги массива B: 
   C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False



Итак, массив содержит те же числа, но расположенные уже по строкам матрицы размера $2 \times 3$. 
Напротив флага `"OWNSDATA"` стоит `False`, т.е. данными он не владеет. Картинка ниже объясняет, что произошло. Создался новый объект с новыми метаданными, который ссылается на тот же массив с данными. Эти новые метаданные указывают, что массив данных следует интерпретировать так, как будто в нём хранится матрица размера $2 \times 3$, записанная по строкам друг за другом.

```{figure} /_static/lecture_specific/numpy/view_b.svg
```

Создадим транспонированную матрицу методом `transpose` и убедимся, что она тоже использует данные исходного массива `A`.

In [24]:
C = B.transpose()
print(C)
print("Массив C пользуется данными массива A?", C.base is A)

[[0 3]
 [1 4]
 [2 5]]
Массив C пользуется данными массива B? False
Массив C пользуется данными массива A? True


Матрица `C` --- транспонирование матрицы `B`. Под неё тоже не выделился новый массив с данными, а только лишь новые метаданные.
В данном случае, метаданные указывают, что в массиве хранится матрица размера $3 \times 2$, которая расположена в памяти таким образом, что первый индекс меняется быстрее второго (т.е. как бы по столбцам).

```{figure} /_static/lecture_specific/numpy/view_c.svg
```

Продемонстрируем, что изменение элементов любого из этих массивов повлияет на остальные.

In [25]:
C[2, 1] = 42
print(A, B, C, sep="\n" + "-"*15 + "\n")

[ 0  1  2  3  4 42]
---------------
[[ 0  1  2]
 [ 3  4 42]]
---------------
[[ 0  3]
 [ 1  4]
 [ 2 42]]
